In [1]:
import phoenix as px
pxc = px.Client()

/Users/jgilhuly/Documents/dev/GitHub/Llama-Researcher/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jgilhuly/Documents/dev/GitHub/Llama-Researcher/.conda/lib/python3.11/site-packages/phoenix/utilities/client.py:45: UserWarning: The Phoenix server (4.32.0) and client (4.36.0) versions are mismatched and may have compatibility issues.
  warnings.warn(


In [2]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents = get_retrieved_documents(pxc, project_name="research_assistant")
retrieved_documents.head()

context.trace_id  \
context.span_id  document_position                                     
35efaab1dd55534b 0                  255d389c1d1edd773fff4748adaab82d   
                 1                  255d389c1d1edd773fff4748adaab82d   
                 2                  255d389c1d1edd773fff4748adaab82d   
                 3                  255d389c1d1edd773fff4748adaab82d   
                 4                  255d389c1d1edd773fff4748adaab82d   

                                                                                input  \
context.span_id  document_position                                                      
35efaab1dd55534b 0                  2024 US presidential election candidates and t...   
                 1                  2024 US presidential election candidates and t...   
                 2                  2024 US presidential election candidates and t...   
                 3                  2024 US presidential election candidates and t...   
                 4                  2024 US presidential election candidates and t...   

                                                                            reference  \
context.span_id  document_position                                                      
35efaab1dd55534b 0                  2024 Presidential Election\nAdvertisement\nWho...   
                 1                  Email *\nFirst Name *\nLast Name\nPlease compl...   
                 2                  Campaign positions\nEstablish “an Economic Bil...   
                 3                  Profile\nSections\ntv\nFeatured\nMore From NBC...   
                 4                  Suarez, who is Cuban American, comes from a po...   

                                    document_score  
context.span_id  document_position                  
35efaab1dd55534b 0                        0.680594  
                 1                        0.658504  
                 2                        0.640260  
                 3                        0.589031  
                 4                        0.586914

In [3]:
from phoenix.evals import (
    OpenAIModel,
    llm_classify,
)
import nest_asyncio
nest_asyncio.apply()

model = OpenAIModel(
    model="gpt-4o",
    temperature=0.0,
)

bias_detection_prompt = """
You are an AI assistant designed to detect bias in documents. Given an original question 
and a document, your task is to analyze the document and determine if there is any bias present. 

Please follow these steps:
1. Read the original question carefully.
2. Read the document provided.
3. Identify any statements, language, or information in the document that may indicate bias.
4. Provide a classification of the document's bias level: 
Unbiased, Somewhat Unbiased, Somewhat Biased, Biased.
5. Provide a detailed explanation for your classification, citing specific parts of the 
document that influenced your decision.

Original Question: {input}
Document: {reference}

Your analysis should be thorough and objective. Please ensure that your explanation 
is clear and concise.

Example response:
************
EXPLANATION: An explanation of your reasoning for the label you chose
LABEL: "bias", "unbiased", "somewhat biased", "somewhat unbiased"
************
"""

In [4]:
bias_classifications = llm_classify(
    dataframe=retrieved_documents, 
    template=bias_detection_prompt, 
    model=model, 
    rails=['Unbiased', 'Biased', 'Somewhat Biased', 'Somewhat Unbiased'],
    provide_explanation=True,
)
bias_classifications["score"] = bias_classifications["label"].map(
    {"unbiased": 1, "somewhat unbiased": 0.75, "somewhat biased": 0.5, "biased": 0}
    )

llm_classify |██████████| 119/119 (100.0%) | ⏳ 00:37<00:00 |  3.20it/s


In [5]:
bias_classifications.head()

label  \
context.span_id  document_position                      
35efaab1dd55534b 0                  somewhat unbiased   
                 1                           unbiased   
                 2                           unbiased   
                 3                    somewhat biased   
                 4                             biased   

                                                                          explanation  \
context.span_id  document_position                                                      
35efaab1dd55534b 0                  The document provides a detailed overview of t...   
                 1                  The document provides a factual and comprehens...   
                 2                  The document provides a factual summary of Jil...   
                 3                  The document exhibits a degree of bias, partic...   
                 4                  The document exhibits a degree of bias in its ...   

                                   exceptions execution_status  \
context.span_id  document_position                               
35efaab1dd55534b 0                         []        COMPLETED   
                 1                         []        COMPLETED   
                 2                         []        COMPLETED   
                 3                         []        COMPLETED   
                 4                         []        COMPLETED   

                                    execution_seconds  score  
context.span_id  document_position                            
35efaab1dd55534b 0                           7.456938   0.75  
                 1                           7.188562   1.00  
                 2                           3.416684   1.00  
                 3                          10.110849   0.50  
                 4                           4.191071   0.00

In [6]:
span_bias_classifications = bias_classifications.copy()

span_bias_classifications["average_score"] = span_bias_classifications.groupby("context.span_id")["score"].transform("mean")

span_bias_classifications["label"] = span_bias_classifications["average_score"].apply(
    lambda x: min({1: "unbiased", 0.75: "somewhat unbiased", 0.5: "somewhat biased", 0: "biased"}.keys(), key=lambda k: abs(k - x))
).map(
    {1: "unbiased", 0.75: "somewhat unbiased", 0.5: "somewhat biased", 0: "biased"}
)

# Combine all rows with the same context.span_id into one row, with explanations being a concatenation of all the explanations
span_bias_classifications = span_bias_classifications.groupby("context.span_id").agg({
    "label": "first",
    "explanation": lambda x: "\n----\n".join(x),
    "exceptions": "first",
    "execution_status": "first",
    "execution_seconds": "mean",
    "score": "mean",
    "average_score": "first"
}).reset_index()
span_bias_classifications.set_index("context.span_id", inplace=True)
span_bias_classifications.head()


TypeError: sequence item 10: expected str instance, NoneType found

In [ ]:
from phoenix.trace import SpanEvaluations, DocumentEvaluations

px.Client().log_evaluations(
    SpanEvaluations(
        dataframe=span_bias_classifications,
        eval_name="Bias Detection",
    ),
    DocumentEvaluations(
        dataframe=bias_classifications,
        eval_name="Relevance",
    ),
)